In [182]:
import bqplot.pyplot as plt
import bqplot as bq
import numpy as np

from IPython.display import display
from ipywidgets import widgets

In [183]:
class ZerosVisualizer:
    def __init__(self, fun, interval):
        self.x0, self.x1 = interval
        self.fun = fun
        self.xn = self.x0

        self.f0 = self.fun(self.x0)
        self.f1 = self.fun(self.x1)
        if self.f0 * self.f1 > 0:
            raise ValueError("The function has no zeros in the given interval")

        self.fn = self.f0

        self.e = self.x1 - self.xn

        self.iterations = 0

        self.x = np.linspace(self.x0, self.x1, 100)
        self.y = self.fun(self.x)

    def fixPoints(self):
        # Corrrects the point for the Algorithm to work
        if np.sign(self.f0) == np.sign(self.f1):
            self.x0 = self.xn
            self.f0 = self.fn
            self.e = self.x1 - self.xn
        elif np.sign(self.f0) < np.sign(self.f1):
            self.x0, self.x1, self.xn = self.x1, self.x0, self.x1
            self.f0, self.f1, self.fn = self.f1, self.f0, self.f1

    def newPoints(self):
        self.fixPoints()

        m = 0.5 * (self.x0 - self.x1)
        self.tol = 2.0 * np.finfo(float).eps * max(abs(self.x1), 1.0)
        if np.isclose(self.f1, 0) or abs(m) < self.tol:
            return "STOP"

        # Bisection
        self.bisectedStep = (self.x0, self.x1 + m, self.x1)
        # IQI Step
        s = self.f1 / self.fn
        if self.x0 == self.xn:
            # Linear interpolation
            p = 0
            q = 1.0 - s
        else:
            # Inverse quadratic interpolation
            q = self.fn / self.f0
            r = self.f1 / self.f0
            p = s * (2.0 * m * q * (q - r) - (self.x1 - self.xn) * (r - 1.0))
            q = (q - 1.0) * (r - 1.0) * (s - 1.0)

        if p > 0.0:
            q = -q
        else:
            p = -p

        if 2.0 * p < 3.0 * m * q - abs(self.tol * q) and p < abs(0.5 * self.e * q):
            d = p / q
        else:
            d = m

        self.iqiStep = (self.x0, self.x1 + d, self.x1)
        if abs(d) < self.tol:
            self.iqiStep = (
                self.x0,
                self.x1 + np.sign(self.x1 - self.x0) * self.tol,
                self.x1,
            )

    def initializeComponents(self):

        self.revertButton = widgets.Button(description="Revert")
        # self.revertButton.on_click(self.revert)

        self.currentSolOut = widgets.Output()  # Current Solution: <x1>
        with self.currentSolOut:
            print("Current Solution: ", self.x1)

        self.helperOut = widgets.Output()  # Next Possible Step: <Bisect/IQI/None>

        # Reset Button
        self.resetButton = widgets.Button(
            description="Reset",
            disabled=False,
            button_style="danger",
            tooltip="Reset",
            icon="undo",
        )
        # self.resetButton.on_click(self.reset)

    def funPlot(self, x, y):
        self.funLine = bq.Lines(
            x=x,
            y=y,
            scales={"x": self.x_sc, "y": self.y_sc},
            colors="#808080",
            name="Real Function",
            display_legend=True,
            labels=["Real Function"],
            enable_move=False,
            enable_add=False,
            line_style="dashed",
        )

    def pointPlot(self, color, marker):
        # Scatter x0,x1 - f0,f1
        self.ogPoints = bq.Scatter(
            x=[self.x0, self.x1],
            y=[self.f0, self.f1],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=[color],
            name="Original Points",
            display_legend=False,
            labels=["Original Points"],
            enable_move=False,
            enable_add=False,
            marker=marker,
        )

        bisected = self.bisectedStep[1]
        self.bisectScatter = bq.Scatter(
            x=[bisected],
            y=[0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=["blue"],
            name="Bisected Point",
            display_legend=True,
            labels=["Bisected Point"],
            enable_move=False,
            enable_add=False,
            marker="diamond",
        )
        self.bisectScatter.on_element_click(self.bisectClick)

        iqi = self.iqiStep[1]
        self.iqiScatter = bq.Scatter(
            x=[iqi],
            y=[0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=["green"],
            name="IQI Point",
            display_legend=True,
            labels=["IQI Point"],
            enable_move=False,
            enable_add=False,
            marker="rectangle",
        )
        self.iqiScatter.on_element_click(self.iqiClick)

    def bisectClick(self, *args):
        self.x0, self.x1, self.xn = self.bisectedStep
        self.fixPoints()
        self.updateValues()

    def iqiClick(self, *args):
        self.e = -self.x1
        self.x0, self.x1, self.xn = self.bisectedStep
        self.e += self.x1
        self.fixPoints()
        self.updateValues()

    def updateValues(self):
        self.f0 = self.fun(self.x0)
        self.f1 = self.fun(self.x1)

        self.fn = self.fun(self.xn)

        self.iterations += 1

        # GET NEXT POINTS AND UPDATE
        # If the new points meet the tolerance, stop the algorithm
        if (
            abs(self.f1) < 10**-10
            or abs(self.e) < self.tol
            or self.newPoints() == "STOP"
        ):
            self.helperOut.clear_output()
            with self.helperOut:
                print("Next Possible Step: None")
            self.bisectScatter.on_element_click(lambda *args: None)
            self.iqiScatter.on_element_click(lambda *args: None)
            return None

        # Update the scatter plot
        self.pointPlot("Black", "cross")  # Plot the points

        # If it gets small enough, adjust the axis to better viewing
        if (self.x1 - self.x0) < 1:
            self.x_sc.min = min(self.x0, self.x1)
            self.x_sc.max = max(self.x0, self.x1)
            self.y_sc.min = min(self.f0, self.f1)
            self.y_sc.max = max(self.f0, self.f1)

            self.funPlot(
                np.linspace(self.x0, self.x1, 100),
                self.fun(np.linspace(self.x0, self.x1, 100)),
            )  # Plot the function

        # Update the plot
        self.Fig.marks = [
            self.Fig.marks[0],
            self.funLine,
            self.Fig.marks[2],
            self.ogPoints,
            self.bisectScatter,
            self.iqiScatter,
        ]

        # Update the current solution
        self.currentSolOut.clear_output()
        with self.currentSolOut:
            print(
                f"Curent Solution: {self.x1}\nValue: {self.f1}\nIterations: {self.iterations}"
            )

        # Update the helper output
        self.helperOut.clear_output()
        with self.helperOut:
            print(
                f"Next Possible Step: {'Bisect' if abs(self.e) < self.tol or abs(self.fn)<=abs(self.f1) else 'IQI'}"
            )

    def runner(self):

        self.x_sc = bq.LinearScale()
        self.y_sc = bq.LinearScale()
        ax_x = bq.Axis(scale=self.x_sc, grid_lines="solid", label="X")
        ax_y = bq.Axis(
            scale=self.y_sc,
            orientation="vertical",
            tick_format="0.2f",
            grid_lines="solid",
            label="Y",
        )
        self.fixPoints()  # Corrects the points for the Algorithm to work
        self.initializeComponents()

        # Plots the function
        self.newPoints()  # Calculate the next points

        with self.helperOut:
            print(
                "Next Possible Step: ",
                "Bisect"
                if abs(self.e) < self.tol or abs(self.fn) < abs(self.f1)
                else "IQI",
            )

        self.pointPlot("Red", "circle")  # Plot the points
        self.funPlot(self.x, self.y)  # Plot the function
        horiLine = bq.Lines(
            x=[self.x0, self.x1],
            y=[0, 0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors="DimGray",
            name="Horizontal Line",
            display_legend=False,
            labels=["Horizontal Line"],
            enable_move=False,
            enable_add=False,
        )

        self.Fig = bq.Figure(
            marks=[
                horiLine,
                self.funLine,
                self.ogPoints,
                self.ogPoints,
                self.bisectScatter,
                self.iqiScatter,
            ],
            axes=[ax_x, ax_y],
            title="Zeros of a Function",
            # legend_location="top-right",
            animation_duration=1000,
        )

        grid = widgets.GridspecLayout(3, 3)
        grid[:2, :2] = self.Fig
        grid[0, 2] = self.currentSolOut
        grid[1, 2] = self.helperOut

        display(grid)  # Display the figure

In [184]:
f = lambda x: (x - 1) * (x - 4)
interval = (0, 3)

from BNumMet.Zeros import fZero

display(fZero(f, interval, iters=True))

zv = ZerosVisualizer(f, interval)

zv.runner()  # Run the visualizer

(1.0, 2)

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='X', scale=LinearScale()), Axis(labe…

In [185]:
class zerosVisualizer2:
    def __init__(self, fun, interval):
        self.f = fun
        self.x0, self.x1 = interval
        self.xn = self.x0

        self.f0 = self.f(self.x0)
        self.f1 = self.f(self.x1)
        self.fn = self.f(self.xn)

        if self.f0 * self.f1 > 0:
            raise ValueError("The function has no zeros in the given interval")

        self.e = self.x1 - self.x0

        self.iterations = 0

        self.previousPoints = []
        self.OrignalPoints = (self.x0, self.x1, self.xn)

    def initializeComponents(self):
        # WIDGETS
        # ==========================================================================
        ## Output Widgets
        self.currentSolOut = widgets.Output()  # Current Solution: <x1>
        # Value: <f1>
        # Iterations: <iterations>
        self.helperOut = widgets.Output()  # Next Possible Step: <Bisect/IQI/None>

        ## Button Widgets
        self.resetButton = widgets.Button(
            description="Reset",
            disabled=False,
            button_style="danger",
            tooltip="Reset",
            icon="undo",
        )
        self.resetButton.on_click(self.reset)

        self.revertButton = widgets.Button(
            description="Revert",
            disable=False,
            button_style="warning",
            tooltip="Revert",
            icon="arrow-left",
        )
        self.revertButton.on_click(self.revert)

        # FIGURE
        # ==========================================================================
        self.x_sc = bq.LinearScale()
        self.y_sc = bq.LinearScale()
        ax_x = bq.Axis(scale=self.x_sc, grid_lines="solid", label="X")
        ax_y = bq.Axis(
            scale=self.y_sc,
            orientation="vertical",
            tick_format="0.2f",
            grid_lines="solid",
            label="Y",
        )

        self.Fig = bq.Figure(
            marks=[],
            axes=[ax_x, ax_y],
            title="Zeros of a Function",
            # legend_location="top-right",
            animation_duration=1000,
        )

        # GRID
        # ==========================================================================
        self.grid = widgets.GridspecLayout(3, 3)
        self.grid[:2, :2] = self.Fig
        self.grid[0, 2] = self.currentSolOut
        self.grid[1, 2] = self.helperOut
        self.grid[2, 2] = widgets.HBox([self.resetButton, self.revertButton])

    def reset(self, *args):
        self.x0, self.x1 = self.OrignalPoints[:2]
        self.xn = self.OrignalPoints[2]
        self.f0 = self.f(self.x0)
        self.f1 = self.f(self.x1)
        self.fn = self.f(self.xn)
        self.e = self.x1 - self.x0
        self.iterations = 0
        self.previousPoints = []
        self.newPoints()
        with self.grid.hold_sync():
            self.updateOutputWidgets()
            self.update()

    def revert(self, *args):
        if len(self.previousPoints) > 0:
            self.x0, self.x1, self.xn = self.previousPoints.pop()
            self.xn = self.x0
            self.f0 = self.f(self.x0)
            self.f1 = self.f(self.x1)
            self.fn = self.f(self.xn)
            self.e = self.x1 - self.x0
            self.iterations -= 1
            self.newPoints()
            with self.grid.hold_sync():
                self.updateOutputWidgets()
                self.update()

    def fixPoints(self):
        if np.sign(self.f0) == np.sign(self.f1):
            self.x0 = self.xn
            self.f0 = self.fn
            self.e = self.x1 - self.xn
        elif np.sign(self.f0) < np.sign(self.f1):
            self.x0, self.x1, self.xn = self.x1, self.x0, self.x1
            self.f0, self.f1, self.fn = self.f1, self.f0, self.f1

    def newPoints(self) -> list:
        self.nextPoints = [None, None]

        m = 0.5 * (self.x0 - self.x1)
        self.tol = 2 * np.finfo(float).eps * max(abs(self.x1), 1)
        if abs(m) < self.tol or np.isclose(self.f1, 0):
            return self.nextPoints

        # BISECTION
        self.nextPoints[0] = self.x1 + m

        # IQI
        self.nextPoints[1] = self.x1 + self.IQI_Step()

        return self.nextPoints

    def nextStep(self):
        return "Bisect" if self.e < self.tol or abs(self.fn) < abs(self.f1) else "IQI"

    def updateOutputWidgets(self):
        self.currentSolOut.clear_output()
        with self.currentSolOut:
            print(
                f"Curent Solution: {self.x1}\nValue: {self.f1}\nIterations: {self.iterations}"
            )
        self.helperOut.clear_output()
        with self.helperOut:
            if all(v is not None for v in self.nextPoints):
                print(f"Next Possible Step: {self.nextStep()}")
            else:
                print("Next Possible Step: None - Already at a Solution")

    def functionPlot(self, x, y, name, color, line_style, display_legend=True):
        return bq.Lines(
            x=x,
            y=y,
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=[color],
            display_legend=display_legend,
            labels=[name],
            enable_move=False,
            enable_add=False,
            line_style=line_style,
        )

    def dotsPlot(
        self, x, y, name, color, marker, clicker=False, val=None, display_legend=True
    ):
        dots = bq.Scatter(
            x=x,
            y=y,
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=[color],
            display_legend=display_legend,
            labels=name,
            enable_move=False,
            enable_add=False,
            marker=marker,
        )
        if clicker:
            dots.val = val
            dots.on_element_click(self.selectPoint)
        return dots

    def selectPoint(self, b, *args):
        if self.nextPoints[b.val] is None:
            return

        self.previousPoints.append((self.x0, self.x1, self.xn))
        self.xn = self.x1
        self.fn = self.f1
        self.x1 = self.nextPoints[b.val]
        self.f1 = self.f(self.x1)
        self.fixPoints()
        self.iterations += 1
        newPoints = self.newPoints()
        with self.grid.hold_sync():
            self.updateOutputWidgets()
            self.update()

    def update(self):
        """
        This function plots everything accordingly to the values inside this class (it does not generate any more points)

        Considerations
            1. If the difference between self.x0 and self.x1 is smaller than 1, we "zoom in" the plot (Original Points not to be plotted)
            2. Else we plot also de original Points and zoom out (X Axis) according to the min/max of all posible points (even the new ones)

        """
        toPlot = []

        # ADJUST AXES
        # ==========================================================================
        toCheck = (
            [self.x0, self.x1] + [self.nextPoints[0], self.nextPoints[1]]
            if all(v is not None for v in self.nextPoints)
            else [self.OrignalPoints[0], self.OrignalPoints[1]]
        )
        self.x_sc.min = min(toCheck)
        self.x_sc.max = max(toCheck)
        if abs(self.x0 - self.x1) > 1:
            self.x_sc.min = min(
                self.x_sc.min, self.OrignalPoints[0], self.OrignalPoints[1]
            )
            self.x_sc.max = max(
                self.x_sc.min, self.OrignalPoints[0], self.OrignalPoints[1]
            )

        # Horizontal Line
        # ==========================================================================
        x = [self.x_sc.min, self.x_sc.max]
        y = [0, 0]
        toPlot.append(self.functionPlot(x, y, "Horizontal Line", "#808080", "solid"))

        # PLOT FUNCTION
        # ==========================================================================
        x = np.linspace(self.x_sc.min, self.x_sc.max, 100)
        y = self.f(x)
        toPlot += [self.functionPlot(x, y, "Real Function", "#808080", "dashed")]
        self.y_sc.min = min(y)
        self.y_sc.max = max(y)

        # PLOT ORIGINAL POINTS
        # ==========================================================================
        x = [self.OrignalPoints[0], self.OrignalPoints[1]]
        y = [self.f(x[0]), self.f(x[1])]
        toPlot += [
            self.dotsPlot(x=x, y=y, name="OriginalPoints", color="red", marker="circle")
        ]

        if all(
            v is not None for v in self.nextPoints
        ):  # If there are new points to plot

            # PLOT CURRENT POINTS
            # ==========================================================================
            x = [self.x0, self.x1]
            y = [self.f0, self.f1]
            toPlot += [
                self.dotsPlot(
                    x=x, y=y, name="Current Points", color="black", marker="cross"
                )
            ]
            # PLOT NEW POINTS
            # ==========================================================================
            ## BISECTION
            x = [self.nextPoints[0]]
            y = [0]
            toPlot += [
                self.dotsPlot(
                    x=x,
                    y=y,
                    name="Bisection",
                    color="blue",
                    marker="diamond",
                    clicker=True,
                    val=0,
                )
            ]
            ## IQI
            x = [self.nextPoints[1]]
            y = [0]
            toPlot += [
                self.dotsPlot(
                    x=x,
                    y=y,
                    name="IQI",
                    color="green",
                    marker="rectangle",
                    clicker=True,
                    val=1,
                )
            ]
        else:
            # PLOT SOLUTION
            # ==========================================================================
            x = [self.x1]
            y = [0]
            toPlot += [
                self.dotsPlot(
                    x=x,
                    y=y,
                    name="Solution",
                    color="black",
                    marker="circle",
                    display_legend=True,
                )
            ]

        self.Fig.marks = toPlot

    def runner(self):
        self.fixPoints()
        self.initializeComponents()
        aux = self.newPoints()
        with self.grid.hold_sync():
            self.updateOutputWidgets()
            self.update()

        return self.grid

    def IQI_Step(self):
        # Interpolation
        m = 0.5 * (self.x0 - self.x1)
        s = self.f1 / self.fn
        if self.x0 == self.xn:
            # Linear interpolation
            p = 2.0 * m * s
            q = 1.0 - s
        else:
            # Inverse quadratic interpolation
            q = self.fn / self.f0
            r = self.f1 / self.f0
            p = s * (2.0 * m * q * (q - r) - (self.x1 - self.xn) * (r - 1.0))
            q = (q - 1.0) * (r - 1.0) * (s - 1.0)

        if p > 0.0:
            q = -q
        else:
            p = -p
        # Is interpolated point acceptable?

        d = p / q if not np.isclose(q, 0) else m

        return d

In [186]:
zv = zerosVisualizer2(f, interval)
display(zv.runner())

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='X', scale=LinearScale(max=3.0, min=…